In [1]:
# load package
import numpy as np
import pandas as pd
from collections import Counter
import lightgbm as lgb
#from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation  import train_test_split
from sklearn import metrics
from sklearn import cross_validation
import pickle

# set path
import os
default_path = "/Users/francislin/temp/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')
#你可以选择修改 ast_note_iteractively kernel 选项来使得 Jupyter 为每一行的变量或语句执行这个操作，以便你可以立即看到多条语句一起输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
with open('all_data_pca134.pickle','rb') as f:
    df= pickle.load(f)

In [17]:
# 區分train test
dataTrain = df[pd.notnull(df['cuisine'])]
dataTest = df[~pd.notnull(df['cuisine'])]
dataTrain.columns
del(df)

Index([     'id', 'cuisine',         0,         1,         2,         3,
               4,         5,         6,         7,
       ...
             124,       125,       126,       127,       128,       129,
             130,       131,       132,       133],
      dtype='object', length=136)

In [18]:
#dataTrain = dataTrain
#去掉y
dataTest = dataTest.drop('cuisine',axis=1)
#去掉key
X_feature = dataTrain.drop(['id','cuisine'],axis=1)
y_t =dataTrain['cuisine']

In [19]:
X_feature.head()

0         1         2         3         4         5         6    \
0  0.410868  1.511810 -1.369136  0.115075 -0.188099 -0.200973 -1.101218   
1  0.034862  0.394947  0.930672 -0.360164 -0.105215 -1.337149 -0.073493   
2  0.332297 -0.650574  0.725765  1.004397  0.015120 -0.228537 -1.393389   
3 -1.303200 -0.633279 -0.821922 -0.810094 -0.343640 -0.631985 -0.309213   
4  0.996157  0.313401  2.253926  0.395730 -0.994413  0.220015 -0.237783   

        7         8         9      ...          124       125       126  \
0 -0.015127 -0.308410 -0.153127    ...    -0.507860  0.008608 -0.222082   
1  0.389514 -0.206608  0.806345    ...    -0.033800  0.186566  0.038546   
2 -0.231589  0.290304 -0.210332    ...     0.100301  0.046994  0.187233   
3  0.482228  0.143415  0.786575    ...     0.043606  0.018030  0.033621   
4 -0.917451  1.639429 -0.105104    ...    -0.241206  0.288573 -0.383483   

        127       128       129       130       131       132       133  
0  0.528199  0.318087 -0.061969 -0.063991 -0.241771  0.266928 -0.022912  
1 -0.371783  0.113266  0.060910 -0.120402 -0.230628 -0.027581  0.050417  
2 -0.073508 -0.066212  0.027006 -0.009036  0.112207  0.126749  0.051914  
3  0.048128  0.075953  0.031704  0.022385  0.022642  0.036007 -0.005594  
4 -0.150991 -0.535665 -0.314691  0.131026 -0.320689  0.230030  0.056896  

[5 rows x 134 columns]

In [20]:
#把上面dataTrain.columns的結果貼到dataTrain，再去掉'Survived'
from sklearn.model_selection import train_test_split
#拿掉feature_importances低的因子
X_train, X_test, y_train, y_test = train_test_split(X_feature, y_t, test_size=0.3, random_state=13)

In [13]:
# Function for Measure Performance
from  sklearn  import  metrics
def measure_performance(X,y,clf, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True, show_roc_auc = True):
    y_pred = clf.predict(X)
    y_predprob = clf.predict_proba(X)[:,1]
    if show_accuracy:
        print ("Accuracy:{0:.3f}".format(metrics.accuracy_score(y,y_pred))),"\n"

    if show_classification_report:
        print("Classification report")
        print(metrics.classification_report(y,y_pred)),"\n"
        
    if show_confusion_matrix:
        print("Confusion matrix")
        print(metrics.confusion_matrix(y,y_pred)),"\n"  
        
    if show_roc_auc:
        print("ROC AUC Score")
        print(metrics.roc_auc_score(y,y_predprob)),"\n"

In [ ]:
X_train_array = X_train.values
X_test_array = X_test.values

In [21]:
%%time
import lightgbm as lgb
lgb = lgb.LGBMClassifier(object='multiclass', num_class=20, max_depth=8,learning_rate=0.25)
lgb_fit=lgb.fit(X_train,y_train)

CPU times: user 2min 33s, sys: 9.28 s, total: 2min 43s
Wall time: 51.4 s


In [22]:
y_pred= lgb_fit.predict(X_test)

In [23]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.36470588, 0.25174825, 0.66093366, 0.72240803, 0.46368715,
       0.48484848, 0.59459459, 0.80832888, 0.29365079, 0.76731152,
       0.5907173 , 0.54742547, 0.61904762, 0.84338546, 0.65633075,
       0.25      , 0.62537463, 0.34871795, 0.72476089, 0.55329949])

0.5585638401799269

0.689097460822928

In [88]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[  72,    1,    2,    1,    3,    4,    0,    0,    0,   20,    0,
           1,    0,   13,    0,    0,   11,    1,    1,    1],
       [   0,   68,    2,    1,    0,   25,    1,    4,   17,   31,    3,
           0,    0,    3,    0,    4,   60,    0,    0,    0],
       [   0,    2,  309,    1,    3,   17,    1,    1,    0,   28,    1,
           0,    0,   20,    1,    0,   70,    2,    0,    0],
       [   1,    0,    2,  672,    6,    9,    2,    3,    0,   11,    0,
          22,   21,   10,    1,    0,   32,    0,   22,   14],
       [  10,    0,    0,   34,  130,    3,    0,    2,    0,   13,    0,
           3,    1,    6,    0,    0,   29,    1,    9,    8],
       [   1,    7,    7,    2,    3,  437,    7,    2,   11,  176,    3,
           1,    0,    6,    3,    9,   68,   11,    1,    1],
       [   0,    1,    0,    0,    0,    5,  240,    7,    0,   71,    0,
           0,    1,    6,    4,    0,    6,    5,    0,    0],
       [   1,    1,    0,    4,    0,    

In [42]:
# Grid Search
print('Start training...')
estimator = lgb.LGBMClassifier(boosting_type='gbdt', object='multiclass', num_class=20, random_state=0) 

param_grid = {
    'learning_rate': [0.11], 
    'num_leaves': [32,64],
    'feature_fraction': [0.3,0.8],
    'bagging_fraction': [0.3,0.9],
    'max_depth':[9],
    #'max_bin': [6],
    'bagging_freq':[4],
    'lambda_l1': [0.1,3,20] , #[44,133] then 44 [44,88] then 44 np.arange(38,44,1) np.arange(16,20,1)
    'lambda_l2': [0.1,1,7] ,#np.arange(11,15,1) then 13
    #'min_split_gain': [0.04], 
    'min_child_weight': [2] 
}

LGBM_grid = GridSearchCV(estimator, param_grid,scoring='accuracy')#scoring='f1_macro'
LGBM_grid.fit(X_train, y_train)
print('Best parameters found by grid search are:', LGBM_grid.best_params_)

# Final Model
evals_result = {} 
print('Start predicting...')
LGBM_grid_final = lgb.LGBMClassifier(boosting_type='gbdt', object='multiclass', num_class=20, random_state=0,
                                     learning_rate = LGBM_grid.best_params_['learning_rate'],
                              num_leaves = LGBM_grid.best_params_['num_leaves'],
                                feature_fraction = LGBM_grid.best_params_['feature_fraction'], 
                             bagging_fraction = LGBM_grid.best_params_['bagging_fraction'],
                             max_depth = LGBM_grid.best_params_['max_depth'],
                              #max_bin = LGBM_grid.best_params_['max_bin'],
                                bagging_freq = LGBM_grid.best_params_['bagging_freq'],
                              lambda_l1 = LGBM_grid.best_params_['lambda_l1'],
                              lambda_l2 = LGBM_grid.best_params_['lambda_l2'],
                            #min_split_gain = LGBM_grid.best_params_['min_split_gain'],
                            min_child_weight = LGBM_grid.best_params_['min_child_weight']
                                    )
LGBM_grid_final_fit = LGBM_grid_final.fit(X_train, y_train)
print("Predicting is over")

Start training...


KeyboardInterrupt: 

In [ ]:
#對test算performance
LGBM_grid_measure = measure_performance(X = X_test, y = y_test, clf = LGBM_grid_final, show_classification_report=True, show_confusion_matrix=True)

In [33]:
#拿全部的dataTrain來建
LGBM_grid_final_fit_all = LGBM_grid_final.fit(X_feature, y_t)

In [34]:
LGBM_grid_final_fit_res =LGBM_grid_final_fit_all.predict(dataTest)

In [18]:
dataTest = dataTest.drop('id',axis=1)

In [35]:
LGBM_grid_final_fit_res

array(['irish', 'southern_us', 'italian', ..., 'italian', 'southern_us',
       'mexican'], dtype=object)

In [36]:
submit=pd.read_csv('./data/sample_submission.csv')

In [37]:
submit['cuisine'] = LGBM_grid_final_fit_res

In [38]:
%pwd
submit.to_csv('submit_to_kaggle.csv', index= False)

"/Users/francislin/JrML/What's Cooking"

In [39]:
submit.cuisine[0]

'irish'

In [27]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=500,max_features=10)
rf_clf.fit(X_train, y_train)

CPU times: user 2min 49s, sys: 1.81 s, total: 2min 51s
Wall time: 2min 52s


In [28]:
y_pred= rf_clf.predict(X_test)

In [29]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.3030303 , 0.0944206 , 0.56456456, 0.6601467 , 0.22695035,
       0.35327103, 0.3490566 , 0.77241379, 0.07142857, 0.68379751,
       0.35233161, 0.49683544, 0.44984802, 0.76533637, 0.28268551,
       0.02684564, 0.57385977, 0.13380282, 0.66666667, 0.36012862])

0.40937102464941455

0.6161903963797871

In [30]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
GBDT = GradientBoostingClassifier(n_estimators=100,max_depth=5,learning_rate=0.1)#深度從3變5沒有比較好＠＠
GBDT.fit(X_train,y_train)

CPU times: user 20min 2s, sys: 5.13 s, total: 20min 7s
Wall time: 20min 13s


In [31]:
y_pred= GBDT.predict(X_test)

In [32]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.29347826, 0.1980198 , 0.63980464, 0.69152542, 0.43501326,
       0.44920525, 0.52406417, 0.76623377, 0.25342466, 0.75115634,
       0.52964427, 0.50886076, 0.57471264, 0.81545702, 0.56265985,
       0.20512821, 0.60767947, 0.35323383, 0.65959703, 0.45346062])

0.513617963345428

0.6549903628592978

In [24]:
%%time
from xgboost import XGBClassifier
Xgb = XGBClassifier()
Xgb.fit(X_train,y_train)

CPU times: user 7min, sys: 870 ms, total: 7min 1s
Wall time: 7min 3s


In [25]:
y_pred= Xgb.predict(X_test)

In [26]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.21428571, 0.10788382, 0.63171036, 0.67710583, 0.31097561,
       0.37453738, 0.48      , 0.73605948, 0.21428571, 0.71224165,
       0.59259259, 0.45737483, 0.52941176, 0.77624242, 0.55696203,
       0.19512195, 0.5410691 , 0.29799427, 0.66173362, 0.42346939])

0.4745528761832576

0.6222240844716332